# Monte Carlo

Now that you have written your own MD code, you have a recipe for evaluating thermodynamic properties of systems in the microcanonical ensemble. We used MD simulations to sample configuration from the ensemble and, since they all had the same probability, we took their arithmetic mean to calculate ensemble averages. But what would you do if you wanted to sample the canonical ensemble? You would have to evaluate Eq.[](ensemble_av_NVT), in which each configuration has a different weight. Effectively, this also means evaluating very **high-dimensional integrals**, since $\textbf x$ is of dimension $6N$ where $N$ is the number of atoms. Here, we will learn about the difficulties of calculating such integrals and of an ingenious alogrithm to evaluate them.

## Evaluation of high-dimensional integrals

As we have seen, the expectation values we encounter in statistical mechanics are of the form 

$$
A = \langle a \rangle_f = \int f(\textbf x) \, a(\textbf{x}) \, \mathrm{d} \textbf{x},
$$ 
where 

$$
f(\textbf x) = \frac{1}{\mathcal Z} \mathcal{F}(\mathcal{H} (\textbf{x}))
$$
is a positive, normalized phase space probability density. From now on we will consider the canonical ensemble, for which $\mathcal{F}(\mathcal{H} (\textbf{x})) = e^{-\beta \mathcal H}$.

How could they be evaluated in practice? One way is to sample $a(\textbf{x}_j)$ on a multi dimensional grid, multiplying each one of the samples by its probability and doing the integration by quadrature,

$$
A_M = \sum_{j=1}^{N_g} f(\textbf{x}_j) a(\textbf{x}_j) \Delta \textbf x = \frac{ \sum_{j=1}^{N_g} e^{-\beta \mathcal{H}(\textbf{x}_j)}\, a(\textbf{x}_j)}{ \sum_{j=1}^{N_g} e^{-\beta \mathcal{H} (\textbf{x}_j) }}.
$$ (quad)

However, this is very, very expensive and the algorithm scales exponentially with the particles. Why? Because we need to evaluate a $6N$ dimensional integral on a grid. Admittedly, if the property we are interested in does not depend on the momenta, $a(\textbf{x}_j) = (\textbf{r}_j)$, then the integration over $\textbf p$ can be done analytically to obtain,

$$
A_M = \frac{ \sum_{j=1}^{N_g} e^{-\beta V(\textbf{r}_j)}\, a(\textbf{r}_j)}{ \sum_{j=1}^{N_g} e^{-\beta V(\textbf{r}_j) }}.
$$ (config_part)

Nevertheless, we remain with a $3N$ dimensional integral. To evalaute it, even if we divide each spatial dimension for every particle into only 10 grid points, we would need to sum over $N_g = 10^{3N}$ points to evaluate the integral. This is completey prohibitive already for a very small number of particles. Monte carlo is a clever way to evaluate such integrals without resorting to qudrature. Moreover, Monte Carlo is a conceptually-different approach for sampling equilibrium distribution functions that does not rely on the dynamics in time of the system to generate the configurations (like MD).

## Random sampling

The first solution to this problem was invented by Metropolis and Ulam in 1949. Their ingenious idea was to use random numbers to evaluate multidimensional integrals instead of a quadrature on a grid. 

They showed that using their approach, it is possible to evalaute integrals using a number of samples that is much smaller than the exponentially growing number of grid points.

In [202]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.special
import time

R = 1
Nsamples = 100000
d = 2
M=10

res = np.array([])
for seed in range(M):

    rng = np.random.default_rng()

    #generate random samples
    p = rng.uniform(0,R, size= (d,Nsamples))
    
    #count how many are inside the n-ball
    inside = ((p**2).sum(axis=0) <= R**2).sum()
    V0 = R**d
    res = np.append(res, inside/Nsamples*V0)
    
    #print( "Result = " + str(inside/Nsamples*V0) )

print("Mean = " + str(res.mean()) ) 
print("STD = " + str(res.std()/np.sqrt(M)) )

#volume of n-ball in first quadrant
voln = math.pi**(d/2) * R**d / scipy.special.gamma(d/2 + 1) / 2**d
print( "Expected = " +  str(voln) )

print("Nsamples = " + str(Nsamples) )
print("10^d grid points = " + str(10**d) )

Mean = 0.785496
STD = 0.0005232651335604136
Expected = 0.7853981633974483
Nsamples = 100000
10^d grid points = 100


In [201]:
from itertools import product
Ng=1000
x = np.linspace(0,R,Ng)

mysum = 0
for point in product(x, repeat=d):
    val = (np.array(point)**2).sum()
    if val <= R**2: mysum+=1

mysum *= (x[1] - x[0])**d 
print(mysum-voln)


0.0009627721126362898


## Importance sampling

We have just established that random (uniform) sampling could be very helpful for multi-dimensional integration, in comparison to numerical quadrature. But there are situations in which uniform sampling could be highly wateful too. Consider the case when $f(\textbf x)$ is localized in some point in the $3N$ dimensional space. Much of our uniform samples will not be counted when evaluating the integral. Can we do better? Yes! If we know how to generate samples that are already distributed according to $f(\textbf x)$.

At this point, we will *assume* that we have an algorithm for obtaining a set of $M$ **independent** samples (configurations of the system) $\textbf x_1 ,..., \textbf x_M$ that are all **identically distributed** according to $f(\textbf x)$ with some mean $\langle a \rangle_f$ and variance $\sigma^2 = \langle a^2 \rangle_f - \langle a \rangle_f^2$. This is by no means a trivial task. It is straightforward only for simple distributions and the form of Monte Carlo that we will study next was designed to address this problem exactly issue. But, for now, we will simply assume such an algorithm exists. If that is the case, we can then evaluate $a(\textbf{x}_j)$ for each sample. 

The central limit theorem (CLT) ensures that if we consider the simple arithmetic mean of the samples as a random variable,

$$
A_M = \frac{1}{M} \sum_{j=1}^{M} a(\textbf{x}_j).
$$ (mean)
It will be normally distributed with the same mean $ \langle a \rangle_f $ as the samples $ \textbf{x}_j $ and a variance 

$$
\sigma^2_M = \frac{\sigma^2}{M}
$$

In other words, the CLT guarantees that the arithmetic mean of the samples converges to ensemble average of $A$ in the limit $M \to \infty$, because its variance from the mean decays like $M^{-1}$. Note that the CLT does not assume a specific form of the distribution $f(\textbf{x})$. This is very powerful, it means that even when our sampling algorithm (dynamic or not) provides samples from a nonuniform distribution, the arithmetic mean will still converge to the true expectation value. The buttom line? We can use the simple arithmetic mean of the samples for the canonical ensemble, just like in the NVE ensemble, as long as we can generate samples from the desired distribution.

We can also use importance sampling if we don't know how to sample from $f(\textbf x)$, but do know how to sample from a different distribution, $h(\textbf x)$, that has much more substantial overlap with $f(\textbf x)$ than the uniform distribution. 

## Metropolis algorithm

We already saw that if we have a way to sample $f(\textbf x)$, we can easily obtain expectation values using importance sampling MC. But it is an unfortunate fact that we have efficient algorithms to sample only relatively simple distributions, such as exponential, Gaussian, uniform etc. 

In a seminal paper in 1953, the Rosenbluths, Tellers and Metropolis (sans spouse) developed an algorithm that can be shown to provide importance sampling from arbitrary distribution function! The importance of this result truly could not be overstated. It is perhaps one of the greatest results of the previous century.



## Acceptance ratio and detailed balance 

## Moves and sampling other ensembles

translation, isothermal-isobaric, grand canonical?